# Installation et importation des librairies:

In [1]:
pip install datasets transformers evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=40c532b79345aabf82fb831241b92b02cf75f083f797db0810a2e709406ceace
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, get_scheduler, pipeline
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
import evaluate
from tqdm.auto import tqdm

2024-04-01 13:38:35.058823: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 13:38:35.058933: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 13:38:35.211299: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data :

In [4]:
raw_datasets = load_dataset("conll2003")

Generating train split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3454 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Preprocessing:

In [5]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [6]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [7]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels,
                                      batched=True,
                                      remove_columns=raw_datasets["train"].column_names)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

# DataLoaders with DataCollator:

In [8]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

train_dataloader = DataLoader(tokenized_datasets["train"],
                              shuffle=True,
                              collate_fn=data_collator,
                              batch_size=8)

eval_dataloader = DataLoader(tokenized_datasets["validation"],
                             collate_fn=data_collator,
                             batch_size=8)

test_dataloader = DataLoader(tokenized_datasets["test"],
                             collate_fn=data_collator,
                             batch_size=8)

# Initialize model:

In [9]:
ner_feature = raw_datasets["train"].features["ner_tags"]
label_names = ner_feature.feature.names
label_names

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, id2label=id2label, label2id=label2id)
model.to(device)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

# Optimizer and Learning Rate Scheduler:

In [10]:
optimizer = Adam(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08)

In [11]:
num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler("linear",
                             optimizer=optimizer,
                             num_warmup_steps=0,
                             num_training_steps=num_training_steps)

# Postprocessing:

In [12]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()
    
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

# Fine tuning:

## Evaluation metric:

In [13]:
metric = evaluate.load("seqeval")

## Training and validation:

In [14]:
training_losses = []
validation_losses = []

progress_bar = tqdm(range(num_train_epochs * len(train_dataloader)))

for epoch in range(num_train_epochs):
    model.train()
    total_train_loss = 0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    average_train_loss = total_train_loss / len(train_dataloader)
    training_losses.append(average_train_loss)
    
    model.eval()
    total_eval_loss = 0
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            loss = outputs.loss
            total_eval_loss += loss.item()
        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        
        true_predictions, true_labels = postprocess(predictions, labels)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    average_eval_loss = total_eval_loss / len(eval_dataloader)
    validation_losses.append(average_eval_loss)
    
    results = metric.compute()
    overalls = {key: results[f"overall_{key}"] for key in ["precision", "recall", "f1", "accuracy"]}
    
    print(f"Epoch {epoch+1}, Training Loss: {average_train_loss:.4f}, Validation Loss: {average_eval_loss:.4f}, "
          f"Precision: {overalls['precision']:.4f}, Recall: {overalls['recall']:.4f}, "
          f"F1: {overalls['f1']:.4f}, Accuracy: {overalls['accuracy']:.4f}")

  0%|          | 0/5268 [00:00<?, ?it/s]

Epoch 1, Training Loss: 0.1021, Validation Loss: 0.0433, Precision: 0.9392, Recall: 0.9255, F1: 0.9323, Accuracy: 0.9890
Epoch 2, Training Loss: 0.0253, Validation Loss: 0.0385, Precision: 0.9468, Recall: 0.9383, F1: 0.9425, Accuracy: 0.9904
Epoch 3, Training Loss: 0.0131, Validation Loss: 0.0393, Precision: 0.9493, Recall: 0.9408, F1: 0.9450, Accuracy: 0.9908


# Evaluation on Test Dataset:

In [15]:
model.eval()

test_losses = []
total_test_loss = 0

with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        
        loss = outputs.loss
        total_test_loss += loss.item()
        
        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        
        true_predictions, true_labels = postprocess(predictions, labels)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    average_test_loss = total_test_loss / len(test_dataloader)
    test_losses.append(average_test_loss)
    
    results = metric.compute()
    overalls = {key: results[f"overall_{key}"] for key in ["precision", "recall", "f1", "accuracy"]}

print(f"Test Loss: {average_test_loss:.4f}, "
      f"Precision: {overalls['precision']:.4f}, "
      f"Recall: {overalls['recall']:.4f}, "
      f"F1: {overalls['f1']:.4f}, "
      f"Accuracy: {overalls['accuracy']:.4f}")

Test Loss: 0.1151, Precision: 0.9180, Recall: 0.9050, F1: 0.9115, Accuracy: 0.9827


# Inference :

In [16]:
token_classifier = pipeline("token-classification",
                            model=model,
                            tokenizer=tokenizer,
                            aggregation_strategy="simple")

text = "My name is Jack and I work with Clara at Meta in Paris."

results = token_classifier(text)

for result in results:
    print(result)

{'entity_group': 'PER', 'score': 0.99755317, 'word': 'Jack', 'start': 11, 'end': 15}
{'entity_group': 'PER', 'score': 0.9933848, 'word': 'Clara', 'start': 32, 'end': 37}
{'entity_group': 'ORG', 'score': 0.8587146, 'word': 'Meta', 'start': 41, 'end': 45}
{'entity_group': 'LOC', 'score': 0.9993374, 'word': 'Paris', 'start': 49, 'end': 54}
